<a href="https://colab.research.google.com/github/Carridl190/Pred_Models/blob/main/Git_BT_Inception01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing import image
from keras import backend as K
import os
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import io
import seaborn as sns
from sklearn.metrics import confusion_matrix
import cv2


In [ ]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32, (3, 3), name = "L1", input_shape=(224,224,3)))
model.add(Activation('relu', name = "L2"))
model.add(MaxPooling2D(pool_size=(2, 2), name = "L3"))

model.add(Conv2D(64, (3, 3), name = "L4"))
model.add(Activation('relu', name = "L5"))
model.add(MaxPooling2D(pool_size=(2, 2), name = "L6"))

model.add(Conv2D(64, (3, 3), name = "L7"))
model.add(Activation('relu', name = "L8"))
model.add(MaxPooling2D(pool_size=(2, 2),name = "L9"))

model.add(Conv2D(128, (3, 3), name = "L10"))
model.add(Activation('relu', name = "L11"))
model.add(MaxPooling2D(pool_size=(2, 2), name = "L12"))

model.add(Flatten(name = "L13"))
model.add(Dense(64, name = "L14"))
model.add(Activation('relu', name = "L15"))
model.add(Dropout(0.5, name = "L16"))
model.add(Dense(1,name = "L17"))
model.add(Activation('softmax', name = "L18"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
#Train from scratch
#Data Augmentation
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_datagen = image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/YourPath_Name',
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary"
)

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    '/YourPath_Name',
    target_size = (224,224),
    batch_size = 32,
    class_mode = "binary"
)

In [ ]:
hist = model.fit(
    train_generator,
    steps_per_epoch=3,
    epochs = 15,
    batch_size=10,
    validation_data = validation_generator,
    validation_steps=2
)

In [ ]:
for layer in model.layers:
    weights = layer.get_weights() # list of numpy arrays

In [ ]:
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['val_accuracy'], label = 'val_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label = 'val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
model.save("BTSeg_DetNet.h5")

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
#Test Images

model = load_model("Your_ModelName")

train_generator.class_indices

In [ ]:
y_actual = []
y_test = []

In [ ]:
for i in os.listdir("YourFile_Path"):
    img = image.load_img("YourFile_Path"+i, target_size = (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    p = model.predict(img)
    #p = model.predict_classes(img)
    y_test.append(p[0,0])
    y_actual.append(1)

In [ ]:
for i in os.listdir("/YourFile_Path"):
    img = image.load_img("/YourFile_Path"+i, target_size = (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    p = model.predict(img)
    #p = model.predict_classes(img)
    y_test.append(p[0,0])
    y_actual.append(0)

In [ ]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)

cm = confusion_matrix(y_actual, y_test)

sns.heatmap(cm, cmap="plasma", annot=True, xticklabels=('Meninglioma','Pituitary'), yticklabels=('Meninglioma','Pituitary'))
plt.ylabel('True Data')
plt.xlabel('Predicted Data')

In [ ]:
#Entrenamiento y validacion se utilizan para saber cuando terminar el entrenamiento
#Matriz de confusion es herramienta gráfica que indica precisiones
#Kaggle concursos

In [ ]:
NORMAL_TEST_PATH = "/YourFile_Path.jpg"
COVID_TEST_PATH = "/YourFile_Path.jpg"

In [ ]:
#To test a single image

img = image.load_img(COVID_TEST_PATH, target_size=(224,224))
plt.imshow(img)
plt.show()

X = image.img_to_array(img)
X = np.expand_dims(X,axis = 0)
images = np.vstack([X])
val = model.predict(images)
if val == 0:
  print("Meninglioma detected")
else:
  print("Pituitary Detected")

# print("Predicted:", decode_predictions(preds, top=1)[0])

In [ ]:
MENINGLIOMA_FLDR_TEST_PATH = "/YourFile_Path"
PITUITARY_FLDER_TEST_PATH = "/YourFile_Path"

In [ ]:
# To test a whole folder with images

for i in os.listdir(MENINGLIOMA_FLDR_TEST_PATH):
  img = image.load_img(PITUITARY_FLDER_TEST_PATH+'//'+ i, target_size=(224,224))
  plt.imshow(img)
  plt.show()

  X = image.img_to_array(img)
  X = np.expand_dims(X,axis = 0)
  images = np.vstack([X])
  val = model.predict(images)
  if val == 0:
    print("Meninglioma detected")
  else:
    print("Pituitary Detected")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
model_builder = keras.applications.xception.Xception
img_size = (299, 299)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "block14_sepconv2_act"
classifier_layer_names = [
    "avg_pool",
    "predictions",
]

# The local path to our target image
img_path = "./Your_image_path.jpg"

display(Image(img_path, width = 224, height = 224))

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Make model
model = model_builder(weights="imagenet")

# Print what the top predicted class is
preds = model.predict(img_array)

# Generate class activation heatmap
heatmap = make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
# We load the original image
img = keras.preprocessing.image.load_img(img_path)
img = keras.preprocessing.image.img_to_array(img)

# We rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# We use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# We use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# We create an image with RGB colorized heatmap
jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * 0.4 + img
superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

# Save the superimposed image
save_path = "img.jpg"
superimposed_img.save(save_path)

# Display Grad CAM
display(Image(save_path, width = 224, height = 224))

In [ ]:
import tensorflow as tf;
print(tf.__version__)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import os
import imutils
import matplotlib.pyplot as plt
import cv2

In [ ]:
new_model = tf.keras.models.load_model('Your_ModelName')
new_model.summary()

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
class GradCAM:
	def __init__(self, model, classIdx, layerName=None):
		# store the model, the class index used to measure the class
		# activation map, and the layer to be used when visualizing
		# the class activation map
		self.model = model
		self.classIdx = classIdx
		self.layerName = layerName
		# if the layer name is None, attempt to automatically find
		# the target output layer
		if self.layerName is None:
			self.layerName = self.find_target_layer()

In [ ]:
	def find_target_layer(self):
		# attempt to find the final convolutional layer in the network
		# by looping over the layers of the network in reverse order
		for layer in reversed(self.model.layers):
			# check to see if the layer has a 4D output
			if len(layer.output_shape) == 4:
				return layer.name
		# otherwise, we could not find a 4D layer so the GradCAM
		# algorithm cannot be applied
		raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

In [ ]:
def compute_heatmap(self, image, eps=1e-8):
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output,
                self.model.output])
        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]
        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)
        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
# resize the heatmap to oringnal X-Ray image size
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
# normalize the heatmap
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
# return the resulting heatmap to the calling function
        return heatmap

In [ ]:
filename = "./Your_imagePath.jpg"
orignal = cv2.imread(filename)
plt.imshow(orignal)
plt.show()

In [ ]:
orig = cv2.cvtColor(orignal, cv2.COLOR_BGR2RGB)
resized = cv2.resize(orig, (224, 224))
dataXG = np.array(resized) / 255.0
dataXG = np.expand_dims(dataXG, axis=0)

In [ ]:
preds = new_model.predict(dataXG)
i = np.argmax(preds[0])
print(i, preds)

In [ ]:
# Old fashioned way to overlay a transparent heatmap onto original image, the same as above
heatmapY = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
heatmapY = cv2.applyColorMap(heatmapY, cv2.COLORMAP_VIRIDIS)  # COLORMAP_JET, COLORMAP_VIRIDIS, COLORMAP_HOT
imageY = cv2.addWeighted(heatmapY, 0.5, orignal, 1.0, 0)
print(heatmapY.shape, orig.shape)
# draw the orignal x-ray, the heatmap, and the overlay together
output = np.hstack([orig, heatmapY, imageY])
fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(np.random.rand(1, 99), interpolation='nearest')
plt.imshow(output)
plt.show()